In [1]:
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/newCFG/hw-memory/benchmarks/spec_example/spec2006_470.lbm/lbm_part.asm"

In [2]:
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 00000000000005e8
.plt 0000000000000600
__cxa_finalize@plt 0000000000000620
__libc_start_main@plt 0000000000000630
__gmon_start__@plt 0000000000000640
abort@plt 0000000000000650
printf@plt 0000000000000660
_start 0000000000000670
call_weak_fn 00000000000006a8
deregister_tm_clones 00000000000006c0
register_tm_clones 00000000000006f0
__do_global_dtors_aux 0000000000000730
frame_dummy 0000000000000778
main 000000000000077c
initializerandommesh 00000000000007dc
swapMesh 000000000000086c
SET_FLAG 00000000000008ac
__libc_csu_init 0000000000000920
__libc_csu_fini 00000000000009a0
_fini 00000000000009a4


In [3]:
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()

tcfg.build_loop_hrchy()
for l in tcfg.loops:
    #print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    #print(l.back_edge.src.name,l.back_edge.dst.name)
    l.back_edge.is_backEdge = True

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 1

for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end


#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.edge_value)


for n in tcfg_nodes:
    print(n.name,"n的值",n.node_value,"n的出边",n.out_num)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)

#for n in tcfg.all_nodes:
#    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
#    print("{} {}".format(n.name, temp))


n0 n3 1.0
n13 n1 0.5
n1 n14 0.5
n14 n2 0.5
n3 n12 1.0
n4 n10 0.5
n5 n8 0.25
n6 n15 0.125
n15 n7 0.125
n7 n8 1
n8 n6 0.125
n8 n9 0.125
n9 n10 1
n10 n5 0.25
n10 n11 0.25
n11 n12 1
n12 n4 0.5
n12 n13 0.5
n0 n的值 1 n的出边 1
n1 n的值 0.5 n的出边 1
n2 n的值 0.5 n的出边 0
n3 n的值 1.0 n的出边 1
n4 n的值 0.5 n的出边 1
n5 n的值 0.25 n的出边 1
n6 n的值 0.125 n的出边 1
n7 n的值 0.125 n的出边 0
n8 n的值 0.25 n的出边 2
n9 n的值 0.125 n的出边 0
n10 n的值 0.5 n的出边 2
n11 n的值 0.25 n的出边 0
n12 n的值 1.0 n的出边 2
n13 n的值 0.5 n的出边 1
n14 n的值 0.5 n的出边 1
n15 n的值 0.125 n的出边 1
n16 n的值 1 n的出边 0
